This notebook is a collection of some things I've learned from GPT. It's a bit tricky without documentation so I hope that this can help

I have tested and run this notebook on MIT supercloud. To take advantage of full memory usage, I recommend running this notebook on a full node vs. a small number of cores.

In [13]:
# For installation instructions on MIT supercloud, see 
# https://gist.github.com/Sam-2727/3c283bd5d5521bde9742db198f03573e
import gpt as g

In [14]:
# some other packages we'll need
import numpy as np
import sys
import io
from os import path
import struct
from typing import Dict, List, Tuple
from xml.etree import ElementTree as ET

GPT is divided into three components: fundamental objects, QCD (i.e. classical non-neural network algorithms and toolkit to calculate other quantities), and machine learning.

Here I will mainly focus on the fundamental objects and QCD side of things, but I think learning the fundamental objects side of GPT is enough to then read and understand the machine learning code directly:
https://github.com/lehner/gpt/tree/8dc512b219149d0362d9278e68cf4ef6733f1d4e/lib/gpt/ml

# Fundamental GPT objects

There are two objects of interest in GPT: grids and lattices. 

Grids are named in reference to the underlying package Grid on which GPT is built. They are more fundamental and encode the type of data we are working with and also some details about how we want to work with that data (mpi layout or checkerboarding).

Lattices are wrapped on top of grid objects. They store specific instances of data we are working with and function in many respects like tensors (although tensors are different gpt objects). They can be indexed in many respects like numpy arrays, although there are a few differences, in particular how data is returned.

First let's create a grid object. The most basic instantiation of gpt grids is providing the spatial dimensions and float precision

For full details of other parameters, see https://github.com/lehner/gpt/blob/8dc512b219149d0362d9278e68cf4ef6733f1d4e/lib/gpt/core/grid.py#L106

In [30]:
# create a new grid of size x=8,y=8,z=8,t=16
# create with double precision floats
newGrid = g.grid([8,8,8,16],precision=g.double)

In [33]:
print(newGrid)

fdimensions = [8, 8, 8, 16]; mpi = [1, 1, 1, 1]; precision = double; checkerboard = full


In [34]:
newGrid

Note that printing a grid in a notebook is more informative than just calling it (which is sometimes erronously thought to be equivalent to printing in Jupyter notebooks).

Now let's create a lattice object. We need to create the lattice on top of our grid, and because lattice objects can be thought of in many respects as tensors, we need to specify what type of tensor we want. Note that because the spatial lattice sites are already specified in the grid object, we don't need to specify these when creating the lattice object.

In [46]:
# here we are creating a matrix object of spin and color.
# So at each point in spacetime we associate a (4x4x3x3)
# tensor (or 12x12 matrix).
newLattice = g.mspincolor(newGrid)

do NOT try printing lattice objects. GPT will try printing our every entry in the lattice. Instead, if you want to see what type of lattice you are working with, just call the object as below:

In [40]:
newLattice

lattice(ot_matrix_spin_color(4,3),double)

In [42]:
print(newLattice.grid) # check that this matches the grid we inputted

fdimensions = [8, 8, 8, 16]; mpi = [1, 1, 1, 1]; precision = double; checkerboard = full


We can index lattice matrix sites as [x,y,z,t]

In [48]:
newLattice[1,2,3,4].array.shape

(4, 4, 3, 3)

Once the lattice is indexed, we convert to a numpy array with the .array attribute, then find the shape of the returned numpy array. We see that the convention is [spin,spin,color,color]

We can access the values at specific lattice sites by accessing the lattice object directly.

Let's look at the color-color matrix associated with [x,y,z,t,s1,s2]=[1,2,3,4,3,2]

In [51]:
newLattice[1,2,3,4,3,2,:,:]

array([[[[[0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j]]]]])

One subtly is that when we access only spacetime indices [x,y,z,t], the object returned is a gpt tensor object which we need to convert to a numpy array, but when accessing also the underlying spin-color indices, the returned object is a numpy array. So newLattice[1,2,3,4,:,:,:,:] and newLattice[1,2,3,4].array return the same thing, but but the second is a bit faster.

Everything is zero now in our instantiation, but we can change that by providing a numpy array.

We need to make sure that our numpy array has the correct data type. Because we said that our data type was g.double, our numpy data type should be np.cdouble

In [64]:
arrayInput = np.reshape(np.linspace(0,1,9,dtype=np.cdouble),(3,3))
print(arrayInput)

[[0.   +0.j 0.125+0.j 0.25 +0.j]
 [0.375+0.j 0.5  +0.j 0.625+0.j]
 [0.75 +0.j 0.875+0.j 1.   +0.j]]


In [65]:
newLattice[1,2,3,4,3,2,:,:]=arrayInput

In [66]:
newLattice[1,2,3,4,3,2,:,:]

array([[[[[0.   +0.j, 0.125+0.j, 0.25 +0.j],
          [0.375+0.j, 0.5  +0.j, 0.625+0.j],
          [0.75 +0.j, 0.875+0.j, 1.   +0.j]]]]])

In [84]:
newLattice[1,2,3,4,0,0,:,:]

array([[[[[0.+0.j, 1.+0.j, 2.+0.j],
          [3.+0.j, 4.+0.j, 5.+0.j],
          [6.+0.j, 7.+0.j, 8.+0.j]]]]])

**Important point:** Converting between numpy arrays and gpt objects is extremely slow, so it is best to work with just gpt objects or just numpy arrays whenever possible

In [67]:
%timeit newLattice[1,2,3,4,3,2,:,:]

1.57 s ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Some more lattice operations (useful for energy calculations)

Fortunately, we can perform most numpy operations on lattice objects, although things are usually a bit more brittle. Often, lattice operations are named

For example, we can multiply two lattices together.

In [97]:
# so we get a nonzero answer
numpyInput = np.reshape(np.arange(0,144,dtype=np.cdouble),(4,4,3,3))
newLattice[1,2,3,4,:,:,:,:]=numpyInput

In [98]:
expressionToEvaluate = newLattice*newLattice
newLatticeSquared = g(expressionToEvaluate)

GPT only evaluates expressions when wrapped with g(), so you can control where in your code you want the computationally intensive evaluation to occur.

The expression evaluation takes place as a tensor sum at each lattice site (unlike default numpy multiplication). So 

$L[x,y,z,t,s_1,s_3,c_1,c_3]=\sum_{s_2,c_2}L[x,y,z,t,s_1,s_2,c_1,c_2]L[x,y,z,t,s_2,s_3,c_2,c_3]$

In [99]:
# what we expect
summedResult = 0
for s in range(0,4):
    for c in range(0,3):
        summedResult+=numpyInput[0,s,0,c]*numpyInput[s,0,c,0]
print(summedResult)

(14802+0j)


In [100]:
#confirming result
newLatticeSquared[1,2,3,4,0,0,0,0]

array([[[[[14802.+0.j]]]]])

It should be mentioned briefly that vectors also exist in GPT, although I haven't used them too much

In [70]:
newLatticeVector = g.vspincolor(newGrid)

In [78]:
g(newLattice*newLatticeVector)

lattice(ot_vector_spin_color(4,3),double)

We'll come back to some more fundamental GPT operations in the course of things, but first:

# Generating propagators

We will load in gauge configurations from previous computations in chroma. We could also compute in gpt, but that would take too long for a tutorial Jupyter notebook.

For a good tutorial on HMC gauge generation for pseudofermions of a two-flavor degenerate mass action, see https://homepages.uni-regensburg.de/~lec17310/teaching/wise2122/chapter13.pdf 

Note that in that tutorial, if wilson fermions are being used, some code should be changed. In particular, we should replace D_m with wilson_clover (as specified below in the propagator computation section), and critically, two_flavor_ratio_evenodd_schur with two_flavor_evenodd_schur and [D_m,D_pv] with D_m.

gauge configurations from chroma have some xml data before containing binary data. When reading them into gpt, we need to skip to the end of the xml and then load the binary data into a numpy array. We then put that numpy array into a lattice object before saving for later usage.

The only thing that might need modifying is that ">c8" command, depending on the data-precision used. See https://numpy.org/doc/stable/reference/arrays.dtypes.html

In [101]:
import numpy as np
import sys
import gpt as g
import io
from os import path
import struct
from typing import Dict, List, Tuple
from xml.etree import ElementTree as ET
def loadGauge(path,size):
    filename = path
    with open(filename, "rb") as f:
        meta: Dict[str, Tuple[int]] = {}
        buffer = f.read(8)
        while buffer != b"" and buffer != b"\x0A":
            assert buffer.startswith(b"\x45\x67\x89\xAB\x00\x01")
            length = (struct.unpack(">Q", f.read(8))[0] + 7) // 8 * 8
            name = f.read(128).strip(b"\x00").decode("utf-8")
            meta[name] = (f.tell(), length)
            f.seek(length, io.SEEK_CUR)
            buffer = f.read(8)
        f.seek(meta["scidac-private-record-xml"][0])
        #previously used scidac_private_record_xml but no longer used (can remove)
        scidac_private_record_xml = ET.ElementTree(
            ET.fromstring(f.read(meta["scidac-private-record-xml"][1]).strip(b"\x00").decode("utf-8"))
        )   
        f.seek(meta["ildg-binary-data"][0])
        ildg_binary_data = f.read(meta["ildg-binary-data"][1])
        # here, all we're doing is finding where the header ends and the binary data we want to read begins
    Lt,Lz,Ly,Lx,Nl,Nc = size
    u_raw2 = (
                np.frombuffer(ildg_binary_data, ">c8")
                .reshape( Lt, Lz, Ly, Lx, Nl, Nc,Nc)
                .astype(">c8")
            )
    # might need to modify "c8" depending on float precision used.
    # also be careful on time, space order. Might be different depending on your usage.
    u_raw3 = u_raw2.astype(np.complex128)
    rng = g.random("test", "vectorized_ranlux24_24_64")
    grid = g.grid([size[3],size[2],size[1],size[0]],precision=g.double)
    U = g.qcd.gauge.random(grid, rng)
    for i in range(0,4):
        gauge_raw = u_raw3[:,:,:,:,i,:,:]
        U[i][:]=np.ravel(gauge_raw)
    return U

We now load a sample gauge configuration. 

**Important**: the order of time, space is reversed in chroma and gpt typical conventions (although of course these are just conventions). When reading the chroma file above, time goes first, so when specifying size for above function, time goes first.

In [102]:
# change filename as needed
i = 0 
loadGaugeTest = loadGauge("sampleConfig.lime",(48,24,24,24,4,3))
# note that time goes first: [t,x,y,z]=[48,24,24,24]

GPT :    9847.934131 s : Initializing gpt.random(test,vectorized_ranlux24_24_64) took 0.204818 s


we can save this gauge configuration in GPT:

In [103]:
g.save("sampleConfigGPT",loadGaugeTest)

GPT :   10090.751025 s : Switching view to [1,1,1,1]/Write
GPT :   10092.705766 s : Wrote 0.0889893 GB at 0.0455256 GB/s (0.0510122 GB/s for distribution, 1.86079 GB/s for checksum, 0.758947 GB/s for writing, 1 views per node)
GPT :   10092.884791 s : Wrote 0.0889893 GB at 0.512146 GB/s (1.3092 GB/s for distribution, 3.19029 GB/s for checksum, 1.14347 GB/s for writing, 1 views per node)
GPT :   10093.069519 s : Wrote 0.0889893 GB at 0.485227 GB/s (1.59818 GB/s for distribution, 3.7069 GB/s for checksum, 0.858527 GB/s for writing, 1 views per node)
GPT :   10093.283637 s : Wrote 0.0889893 GB at 0.42336 GB/s (1.01458 GB/s for distribution, 2.04426 GB/s for checksum, 1.12791 GB/s for writing, 1 views per node)
GPT :   10093.342481 s : Completed writing sampleConfigGPT in 2.61559 s


GPT has stored the field configuration with double precision:

In [108]:
print(path.getsize("sampleConfig.lime"))
print(path.getsize("sampleConfigGPT/00/0000000000.field"))

191116392
382206172


# Calculating Propagators

We compute the propagator corresponding to our gauge field

We specifiy parameters for our wilson_clover action. 

These can be read from the chroma xml header or specified in the gauge generation to begin with

In [117]:

mass = -0.2450
csw = 1.24930970916466
parameters = {
                "mass":mass,
                "boundary_phases":[1,1,1,-1],
                "csw_t":csw,
                "csw_r":csw,
                "xi_0":1,
                "nu":1,
                "isAnisotropic":False
                }

The chroma header for our gauge configurations specifies that one smearing step of 0.125 is applied. We create a smearing operator sm and then apply U to it. In general this is how objects in gauge work. First we need to instantiate a specific instance of the object which we call an operator, and then apply the operator to the object of interest

In [110]:

sm = g.qcd.gauge.smear.stout(rho=0.125)
U = sm(loadGaugeTest)

Now we instantiate our wilson_clover action term. If in doubt of the parameters of a function, it's best to look at the raw code. In this case:
https://github.com/lehner/gpt/blob/8dc512b219149d0362d9278e68cf4ef6733f1d4e/lib/gpt/qcd/fermion/wilson.py#L75

In [118]:
D_m = g.qcd.fermion.wilson_clover(U, parameters)

There are a lot of different methods for propagator inversion in gpt. In particular, some interesting things have been done with multigrid and machine learning. I just use an even odd preconditioner and the fgcr inversion algorithm (but without SAP like is done in some cases). This works well enough for large lattice spacing

In [111]:
# defining some shortcuts
inv = g.algorithms.inverter
inv_pc = inv.preconditioned
pc = g.qcd.fermion.preconditioner



First we instantiate our inverter algorithm, specifying our tolerance ($10^{-12}$), maximum number of iterations, and restart length (parameter relevant to details of inverter algorithm)

In [112]:
fgcr = inv.fgcr({"eps": 1e-12, "maxiter": 1024, "restartlen": 8})

We can now create our inversion operator with even odd preconditioning

In [119]:
inv = inv_pc(pc.eo2(), fgcr)
inv_w = inv(D_m)

We take the grid corresponding to our gauge field and create a lattice object that will store our propagator. We then add a point source at (x,y,z,t)=(0,0,0,0). Alternatively we could've created a wall source or smeared.

Search gpt code for "gpt.create" for other options (e.g. g.create.wall).

In [ ]:
grid = U2[0].grid
src = g.mspincolor(grid)
src[:]=0
g.create.point(src, (0,0,0,0))
prop = g(inv_w * src)

# Correlation functions